In [29]:
from pymongo import MongoClient
from pymongo import errors
from pprint import pprint
import numpy as np

In [30]:
def connect_database():
    client = MongoClient('localhost', 27017)
    return client

In [31]:
client = connect_database()

In [32]:
client.list_database_names()

['Kitchen_test',
 'Perfume_Database',
 'WorldDevelopmentIndicators',
 'admin',
 'config',
 'local']

In [33]:
perfume_database = client.Perfume_Database

In [34]:
pprint(perfume_database.list_collection_names())

['Cleared_Crawled_Perfumes',
 'Scraped_Perfumes',
 'Extra01',
 'Extra03',
 'Perfumes',
 'Extra02',
 'Fragrances',
 'Crawled_Perfumes']


In [35]:
fragrances = perfume_database.Fragrances
extra01 = perfume_database.Extra01

# Compare the two collection

In [36]:
pprint(fragrances.find_one())

{'_id': ObjectId('65a54ac72b32912eefb9ea54'),
 'base notes': ['Civet', 'Resins', 'Musk'],
 'company': 'Bogue',
 'description': 'Maai by Bogue is a Floral Woody Musk fragrance for women and '
                'men. Maai was launched in 2014. The nose behind this '
                'fragrance is Antonio Gardoni. Top notes are Aldehydes and '
                'Tuberose; middle notes are Jasmine and Rose; base notes are '
                'Civet, Resins and Musk. .',
 'gender': 'for women and men',
 'gender_vote': {'average': 0.2, 'votes_count': 130},
 'image': 'https://fimgs.net/mdimg/perfume/375x500.26681.jpg',
 'longevity': {'eternal': 69,
               'long lasting': 140,
               'moderate': 28,
               'very weak': 23,
               'weak': 9},
 'main accords': {'aldehydic': 71.419,
                  'amber': 70.8567,
                  'animalic': 80.9679,
                  'balsamic': 44.2436,
                  'fresh': 57.3066,
                  'musky': 100.0,
        

In [37]:
pprint(extra01.find_one())

{'Brand': 'Indult',
 'Description': ' Rapa Nui for sugar, Tihota is, quite simply, The One. The '
                "One that will call to you every moment you're not smelling "
                "it, The One that you've only had hints of in other vanilla "
                'perfumes, The One that lasts and lasts, The One that has '
                'perfectly captured the essence of the pure nature of fresh '
                'vanilla beans and has harnessed it beyond your wildest '
                'dreams. It inspires obsessive devotion with its fragrance of '
                'smooth, vanilla bean pods dipped in honeyed water and left to '
                'steep. The result is the pure magic, an unbridled vanilla, '
                "sweet, raw and achingly desirable. Of course, Tihota isn't "
                'the first perfume to focus on the dark sweetness of '
                'vanilla... perfumery is filled with vanillas, but this smells '
                'like the dream of a master perfu

# Integration

In [38]:
integration_pipeline = []

Sample transition to Fragrances

In [39]:
translation_doc = {}

In [40]:
translation_doc["description"] = "$Description"

In [41]:
translation_doc["company"] = "$Brand"

In [42]:
translation_doc["notes"] = "$Notes"

In [43]:
translation_doc["image"] = "$Image URL"

In [44]:
translation_doc["name"] = "$Name"

In [45]:
translation_doc

{'description': '$Description',
 'company': '$Brand',
 'notes': '$Notes',
 'image': '$Image URL',
 'name': '$Name'}

The translation will be used in a project stage in order to do the translation

In [46]:
translate_columns_stage = {
    "$project":translation_doc
}

In [47]:
integration_pipeline.append(translate_columns_stage)

In [48]:
integration_pipeline

[{'$project': {'description': '$Description',
   'company': '$Brand',
   'notes': '$Notes',
   'image': '$Image URL',
   'name': '$Name'}}]

test the translation using aggregation

In [49]:
extra01.aggregate(integration_pipeline).next()

{'_id': ObjectId('65c3d3942be6a0a6d64f5eb6'),
 'description': " Rapa Nui for sugar, Tihota is, quite simply, The One. The One that will call to you every moment you're not smelling it, The One that you've only had hints of in other vanilla perfumes, The One that lasts and lasts, The One that has perfectly captured the essence of the pure nature of fresh vanilla beans and has harnessed it beyond your wildest dreams. It inspires obsessive devotion with its fragrance of smooth, vanilla bean pods dipped in honeyed water and left to steep. The result is the pure magic, an unbridled vanilla, sweet, raw and achingly desirable. Of course, Tihota isn't the first perfume to focus on the dark sweetness of vanilla... perfumery is filled with vanillas, but this smells like the dream of a master perfumer who was obsessed with vanilla and was finally given free rein to worship the note with no apologies. It is, without a sliver of a doubt, the finest pure vanilla we've ever had the honor of carrying,

Now that translation is sucessful, we must merge it with the main dataset.
For doing so a merge stage will be used.

In [50]:
merge_stage = {"$merge": {"into": "Fragrances"}}

In [51]:
integration_pipeline.append(merge_stage)

In [52]:
pprint(integration_pipeline)

[{'$project': {'company': '$Brand',
               'description': '$Description',
               'image': '$Image URL',
               'name': '$Name',
               'notes': '$Notes'}},
 {'$merge': {'into': 'Fragrances'}}]


Now the pipline is ready and will be applied. but before aggregation we'll get a count from the main dataset to compare it with the after.

In [53]:
fragrances.count_documents({})

4563

In [54]:
extra01.aggregate(integration_pipeline)

In [55]:
fragrances.count_documents({})

4563

In [56]:
client.close()